<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

#  Multi-Query-Retriever

This method can automatically use a ChatModel to make slight variations of your initial query to help attempt overcome any issues with cosine similarity distances, this allows you to phrase things more like a general query question rather than a pure document similarity look-up.

In [ ]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

### Loader

In [ ]:
loader = WikipediaLoader(query='MKUltra')
documents = loader.load()

In [ ]:
len(documents)

### Split Documents

In [ ]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

### OpenAI Connection for Embeddings

In [ ]:
import os
f = open('C:\\Users\\Marcial\\Desktop\\desktop_openai.txt')
os.environ['OPENAI_API_KEY'] = f.read()
embedding_function = OpenAIEmbeddings()

In [ ]:
# docs

### Embed Documents for ChromaDB

In [ ]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function,persist_directory='./mk_ultra')
db.persist()

### Use Chat Model to Multi Query

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
question="When was this declassified?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db.as_retriever(),llm=llm)

In [ ]:
# Set logging for the queries
import logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [ ]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)

In [ ]:
len(unique_docs)

In [ ]:
print(unique_docs[0].page_content)